###### I have realized that keeping these many columns will not make a general model which can work on any used car data, Instead I should be more focused on general features which are mostly available on all website & train model on those columns --- NEXT: figure out those columns which are very common among most of websites

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(r"H:\CampusX_DS\week43 - My Projects Aug 2024\used_car_price_prediction\Used-Car-Price-Prediction\src\notebook\data\raw_data\cars24_final_data_5.xlsx")

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df.drop(columns=['content.appointmentId'], inplace=True)

In [5]:
df.isnull().sum()

content.make                                   0
content.model                                  0
content.variant                                0
content.year                                   0
content.transmission                           0
content.bodyType                               0
content.fuelType                               0
content.ownerNumber                            0
content.odometerReading                        0
content.onRoadPrice                            0
content.insuranceType                          0
content.duplicateKey                           0
Airbags                                      286
ABSAntilockBrakingSystem                       0
InstrumentPanelType                          635
NumberOfSpeakers                             346
AirConditioner                                 0
PowerWindowsFront                              0
PowerWindowsRear                               0
SteeringWheelMaterial                          0
ParkingAssistRear   

In [6]:
selected_features = ["content.make","content.model","content.variant","content.year","content.transmission","content.bodyType","content.fuelType","content.ownerNumber","content.odometerReading","content.onRoadPrice","content.insuranceType","content.duplicateKey","Airbags","ABSAntilockBrakingSystem","AirConditioner","Displacementcc","GroundClearancemm","SeatingCapacity","Bootspacelitres","FueltankCapacitylitres","MaxPowerbhp","MaxPowerrpm","MaxTorqueNm","defects","repainted","Left Front Tyre","Right Front Tyre","Left Rear Tyre","Right Rear Tyre","Spare Tyre","content.fitnessUpto_months_remaining","content.insuranceExpiry_months_remaining",]

In [7]:
len(selected_features)

32

In [8]:
df = df[selected_features]

In [9]:
df[selected_features].isnull().sum()

content.make                                  0
content.model                                 0
content.variant                               0
content.year                                  0
content.transmission                          0
content.bodyType                              0
content.fuelType                              0
content.ownerNumber                           0
content.odometerReading                       0
content.onRoadPrice                           0
content.insuranceType                         0
content.duplicateKey                          0
Airbags                                     286
ABSAntilockBrakingSystem                      0
AirConditioner                                0
Displacementcc                                0
GroundClearancemm                           530
SeatingCapacity                               0
Bootspacelitres                             491
FueltankCapacitylitres                        0
MaxPowerbhp                             

### Impute missing values using 3 techniques
1. mean mode
2. median mode
3. knn

In [10]:
import pandas as pd
from sklearn.impute import KNNImputer

In [11]:
# Mean & Mode Imputation
mean_mode_df = df.copy()

# Impute numerical columns with mean
mean_cols = mean_mode_df.select_dtypes(include=['float64', 'int64']).columns
mean_mode_df[mean_cols] = mean_mode_df[mean_cols].fillna(mean_mode_df[mean_cols].mean())

# Impute categorical columns with mode
mode_cols = mean_mode_df.select_dtypes(include=['object']).columns
mean_mode_df[mode_cols] = mean_mode_df[mode_cols].fillna(mean_mode_df[mode_cols].mode().iloc[0])

In [12]:
# Median & Mode Imputation
median_mode_df = df.copy()

# Impute numerical columns with median
median_cols = median_mode_df.select_dtypes(include=['float64', 'int64']).columns
median_mode_df[median_cols] = median_mode_df[median_cols].fillna(median_mode_df[median_cols].median())

# Impute categorical columns with mode
median_mode_df[mode_cols] = median_mode_df[mode_cols].fillna(median_mode_df[mode_cols].mode().iloc[0])

In [13]:
# KNN Imputation
knn_df = df.copy()

# Initialize KNN Imputer
imputer = KNNImputer(n_neighbors=5)

# Impute using KNN
knn_imputed = imputer.fit_transform(knn_df.select_dtypes(include=['float64', 'int64']).fillna(-1))
knn_df[knn_df.select_dtypes(include=['float64', 'int64']).columns] = knn_imputed

# Categorical columns imputed with mode
knn_df[mode_cols] = knn_df[mode_cols].fillna(knn_df[mode_cols].mode().iloc[0])

In [14]:
print(mean_mode_df.isnull().sum().sum())
print(median_mode_df.isnull().sum().sum())
print(knn_df.isnull().sum().sum())

0
0
0


### Data Pre Processing

### 1. Data Preprocessing

#### A. Encoding
- **One-Hot Encoding:** Useful for nominal categorical variables. Creates binary columns for each category.
- **Label Encoding:** Best for ordinal variables where the order matters.

#### B. Scaling
- **Standardization (Z-score scaling):** Centers the data around zero with a standard deviation of one. Useful for algorithms that assume normally distributed data.
- **Min-Max Scaling:** Scales features to a [0, 1] range. Suitable for algorithms sensitive to the magnitude of values.

#### C. Transformations
- **Log Transformation:** Helps to address skewness, particularly for positively skewed data.
- **Square Root Transformation:** Also effective for reducing skewness.
- **Box-Cox or Yeo-Johnson Transformations:** Useful for normalizing data, especially if you want to maintain zero or negative values.

In [15]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import PowerTransformer

In [16]:
def preprocess_dataframe(df):
    # Separate numerical and categorical columns
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cat_cols = df.select_dtypes(include=['object']).columns

    # One-Hot Encoding
    ohe = OneHotEncoder(sparse_output=False, drop='first')
    ohe_data = ohe.fit_transform(df[cat_cols])
    ohe_df = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out(cat_cols))
    
    # Concatenate OHE with numerical data
    processed_df = pd.concat([df[num_cols].reset_index(drop=True), ohe_df.reset_index(drop=True)], axis=1)

    # Z-score Standardization
    scaler = StandardScaler()
    processed_df[num_cols] = scaler.fit_transform(processed_df[num_cols])

    # Yeo-Johnson Transformation
    pt = PowerTransformer(method='yeo-johnson')
    processed_df[num_cols] = pt.fit_transform(processed_df[num_cols])

    return processed_df

In [17]:
# Process each DataFrame
mean_mode_processed = preprocess_dataframe(mean_mode_df)
median_mode_processed = preprocess_dataframe(median_mode_df)
knn_processed = preprocess_dataframe(knn_df)

In [18]:
mean_mode_processed.head()

,content.year,content.ownerNumber,content.odometerReading,content.onRoadPrice,content.duplicateKey,Airbags,ABSAntilockBrakingSystem,AirConditioner,Displacementcc,GroundClearancemm,...,content.bodyType_Sedan,content.fuelType_Diesel,content.fuelType_Petrol,content.insuranceType_Comprehensive,content.insuranceType_Zero Depreciation,Left Front Tyre_WARN,Right Front Tyre_WARN,Left Rear Tyre_WARN,Right Rear Tyre_WARN,Spare Tyre_WARN
0,1.808470,-0.554813,-1.802848,-0.367671,-0.605253,0.052918,0.514443,0.019181,-1.103873,0.653592,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.489632,1.798449,-0.051178,-0.466700,-0.605253,0.052918,0.514443,0.019181,-0.003743,-0.380284,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.585372,-0.554813,0.636454,-0.432438,-0.605253,0.052918,0.514443,0.019181,-0.013191,-0.734758,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,1.347206,-0.554813,-0.364180,-1.008481,-0.605253,0.052918,0.514443,0.019181,-0.013191,0.520395,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.347206,-0.554813,-0.947749,0.828622,1.652201,0.052918,0.514443,0.019181,-0.003743,1.843029,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:
median_mode_processed.head()

,content.year,content.ownerNumber,content.odometerReading,content.onRoadPrice,content.duplicateKey,Airbags,ABSAntilockBrakingSystem,AirConditioner,Displacementcc,GroundClearancemm,...,content.bodyType_Sedan,content.fuelType_Diesel,content.fuelType_Petrol,content.insuranceType_Comprehensive,content.insuranceType_Zero Depreciation,Left Front Tyre_WARN,Right Front Tyre_WARN,Left Rear Tyre_WARN,Right Rear Tyre_WARN,Spare Tyre_WARN
0,1.808470,-0.554813,-1.802848,-0.367671,-0.605253,0.076992,0.514443,0.019181,-1.103873,0.838266,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.489632,1.798449,-0.051178,-0.466700,-0.605253,0.076992,0.514443,0.019181,-0.003743,-0.182651,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.585372,-0.554813,0.636454,-0.432438,-0.605253,0.076992,0.514443,0.019181,-0.013191,-0.580036,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,1.347206,-0.554813,-0.364180,-1.008481,-0.605253,0.076992,0.514443,0.019181,-0.013191,0.718211,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.347206,-0.554813,-0.947749,0.828622,1.652201,0.076992,0.514443,0.019181,-0.003743,1.791873,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [20]:
knn_processed.head()

,content.year,content.ownerNumber,content.odometerReading,content.onRoadPrice,content.duplicateKey,Airbags,ABSAntilockBrakingSystem,AirConditioner,Displacementcc,GroundClearancemm,...,content.bodyType_Sedan,content.fuelType_Diesel,content.fuelType_Petrol,content.insuranceType_Comprehensive,content.insuranceType_Zero Depreciation,Left Front Tyre_WARN,Right Front Tyre_WARN,Left Rear Tyre_WARN,Right Rear Tyre_WARN,Spare Tyre_WARN
0,1.808470,-0.554813,-1.802848,-0.367671,-0.605253,0.19693,0.514443,0.019181,-1.103873,0.539264,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.489632,1.798449,-0.051178,-0.466700,-0.605253,0.19693,0.514443,0.019181,-0.003743,0.098860,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.585372,-0.554813,0.636454,-0.432438,-0.605253,0.19693,0.514443,0.019181,-0.013191,0.001779,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,1.347206,-0.554813,-0.364180,-1.008481,-0.605253,0.19693,0.514443,0.019181,-0.013191,0.459906,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.347206,-0.554813,-0.947749,0.828622,1.652201,0.19693,0.514443,0.019181,-0.003743,1.863268,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
print(mean_mode_processed.shape)
print(median_mode_processed.shape)
print(knn_processed.shape)

(2719, 733)
(2719, 733)
(2719, 733)


### Find Base Accuracy before Feature Selection

In [22]:
# List of dataframes (df1, df2, df3)
datasets = [mean_mode_processed, median_mode_processed, knn_processed]

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import statsmodels.api as sm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import warnings; warnings.filterwarnings('ignore')

# Define a list of algorithms grouped as mentioned
linear_regression_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet()
}

non_linear_regression_models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Machine (SVR)': SVR(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Gaussian Process': GaussianProcessRegressor()
}

advanced_regression_models = {
    'XGBoost': xgb.XGBRegressor(objective='reg:squarederror'),
    'LightGBM': lgb.LGBMRegressor(),
    'CatBoost': CatBoostRegressor(learning_rate=0.1, iterations=500, depth=6, silent=True)
}

# Deep Learning Model
def build_keras_nn(X_train, y_train):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Bayesian Regression Model
bayesian_regression_models = {
    'Bayesian Linear Regression': sm.OLS
}

# Combine all models into one dictionary
all_models = {
    **linear_regression_models,
    **non_linear_regression_models,
    **advanced_regression_models,
    'Keras Neural Network': build_keras_nn,
    **bayesian_regression_models
}

# Function to apply all regression models to a dataset using cross-validation
def apply_regression_models(data, target_column):
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    # Preprocessing: Convert categorical variables to dummy variables, and scale features
    X = pd.get_dummies(X)  # Convert categorical variables to dummy variables
    X = StandardScaler().fit_transform(X)  # Standard scaling
    
    # Dictionary to hold results
    results = []
    
    # Apply each model using cross-validation
    for name, model in all_models.items():
        try:
            if name == 'Keras Neural Network':
                # Keras model needs to be fit on the entire dataset
                build_keras_nn(X, y)
                y_pred = model.predict(X)
                mse = mean_squared_error(y, y_pred)
                r2 = r2_score(y, y_pred)
            elif name == 'Bayesian Linear Regression':
                X_with_const = sm.add_constant(X)
                model_ols = model(X_with_const, y).fit()
                y_pred = model_ols.predict(X_with_const)
                mse = mean_squared_error(y, y_pred)
                r2 = r2_score(y, y_pred)
            else:
                # Use cross-validation
                scores = cross_val_score(model, X, y, scoring='r2', cv=5)
                r2 = np.mean(scores)
                mse = -np.mean(cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5))
            
            # Store result
            results.append({
                'Algorithm': name,
                'MSE': mse,
                'R²': r2
            })
        except Exception as e:
            print(f"Error occurred while processing model {name}: {e}")
    
    # Return the results as a DataFrame, if any results exist
    if results:
        return pd.DataFrame(results)
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no results


# Loop over datasets
results_all_datasets = []

for idx, dataset in enumerate(datasets, start=1):
    try:
        print(f"Applying models to dataset {idx}")
        # Apply regression models on the current dataset
        target_column = 'content.onRoadPrice'
        results_df = apply_regression_models(dataset, target_column)

        # Check if results_df is not empty before appending
        if not results_df.empty:
            # Add dataset identifier to the results
            results_df['Dataset'] = f'dataset_{idx}'
            # Append results to the final list
            results_all_datasets.append(results_df)
        else:
            print(f"No results for dataset {idx}")
    except Exception as e:
        print(f"Error applying models to dataset {idx}: {e}")

# Check if there are any results to concatenate
if results_all_datasets:
    # Combine results of all datasets into one DataFrame
    final_results = pd.concat(results_all_datasets, ignore_index=True)

    # Sort the results by R² (or MSE, depending on your preference)
    final_results_sorted = final_results.sort_values(by='R²', ascending=False)

    # Display the sorted results, including the dataset name
    print(final_results_sorted[['Dataset', 'Algorithm', 'R²', 'MSE']])
else:
    print("No results to concatenate.")

Applying models to dataset 1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1034
[LightGBM] [Info] Number of data points in the train set: 2175, number of used features: 94
[LightGBM] [Info] Start training from score -0.006425
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.310475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1055
[LightGBM] [Info] Number of data points in the train set: 2175, number of used features: 97
[LightGBM] [Info] Start training from score 0.021344
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001767 seconds.
You can set `force_col_wise=true` to remove the overhead

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003002 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1034
[LightGBM] [Info] Number of data points in the train set: 2175, number of used features: 94
[LightGBM] [Info] Start training from score -0.006425
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1055
[LightGBM] [Info] Number of data points in the train set: 2175, number of used features: 97
[LightGBM] [Info] Start training from score 0.021344
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough

In [24]:
final_results_sorted[['Dataset', 'Algorithm', 'R²', 'MSE']]

,Dataset,Algorithm,R²,MSE
33,dataset_3,XGBoost,9.078824e-01,9.379251e-02
21,dataset_2,XGBoost,9.075182e-01,9.419074e-02
9,dataset_1,XGBoost,9.069992e-01,9.482459e-02
5,dataset_1,Random Forest,9.037043e-01,9.653947e-02
17,dataset_2,Random Forest,9.036353e-01,9.637460e-02
29,dataset_3,Random Forest,9.025893e-01,9.723099e-02
35,dataset_3,CatBoost,9.015620e-01,1.000552e-01
25,dataset_3,Ridge Regression,9.011779e-01,1.002095e-01
1,dataset_1,Ridge Regression,9.008823e-01,1.004977e-01
13,dataset_2,Ridge Regression,9.008283e-01,1.005520e-01


### Feature Selection

#### Corelation

In [25]:
def filter_dataframe_by_correlation(df, target_variable, correlation_threshold):
    """
    Filter the DataFrame to retain only features that have a correlation
    above the specified threshold with the target variable.
    
    Parameters:
    df (pd.DataFrame): The original DataFrame.
    target_variable (str): The name of the target variable.
    correlation_threshold (float): The correlation threshold for feature selection.
    
    Returns:
    pd.DataFrame: A filtered DataFrame with only the selected features.
    """
    # Calculate the correlation matrix
    corr_matrix = df.corr()
    
    # Get the correlation of each feature with the target variable
    target_corr = corr_matrix[target_variable]
    
    # Select features with correlation above the threshold
    selected_features = target_corr[abs(target_corr) > correlation_threshold].index.tolist()
    
    # Create and return the filtered DataFrame
    return df[selected_features]

In [26]:
target_variable = 'content.onRoadPrice'
correlation_threshold = 0.3  # Set your desired threshold

# List of processed DataFrames
datasets = [mean_mode_processed, median_mode_processed, knn_processed]

# Create filtered DataFrames using the function
filtered_dfs = []
for df in datasets:
    filtered_df = filter_dataframe_by_correlation(df, target_variable, correlation_threshold)
    filtered_dfs.append(filtered_df)

In [27]:
filtered_dfs[0].columns

Index(['content.year', 'content.onRoadPrice', 'Airbags',
       'ABSAntilockBrakingSystem', 'Displacementcc', 'GroundClearancemm',
       'Bootspacelitres', 'FueltankCapacitylitres', 'MaxPowerbhp',
       'MaxTorqueNm', 'content.fitnessUpto_months_remaining',
       'content.make_Maruti', 'content.bodyType_SUV',
       'content.fuelType_Diesel'],
      dtype='object')

### Embeded Technique feature selection for all columns

In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

In [29]:
def lasso_feature_selection(X, y, alpha=0.01):
    lasso = Lasso(alpha=alpha)
    lasso.fit(X, y)
    return X.columns[lasso.coef_ != 0]

def ridge_feature_selection(X, y, alpha=1.0):
    ridge = Ridge(alpha=alpha)
    ridge.fit(X, y)
    return X.columns[ridge.coef_ != 0]

def elastic_net_feature_selection(X, y, alpha=0.01, l1_ratio=0.5):
    elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    elastic_net.fit(X, y)
    return X.columns[elastic_net.coef_ != 0]

def random_forest_feature_selection(X, y):
    rf = RandomForestRegressor()
    rf.fit(X, y)
    return X.columns[rf.feature_importances_ > 0.01]  # Adjust threshold as needed

def gradient_boosting_feature_selection(X, y):
    gb = GradientBoostingRegressor()
    gb.fit(X, y)
    return X.columns[gb.feature_importances_ > 0.01]  # Adjust threshold as needed

def xgboost_feature_selection(X, y):
    xgb = XGBRegressor()
    xgb.fit(X, y)
    return X.columns[xgb.feature_importances_ > 0.01]  # Adjust threshold as needed

def lightgbm_feature_selection(X, y):
    lgbm = LGBMRegressor()
    lgbm.fit(X, y)
    return X.columns[lgbm.feature_importances_ > 0.01]  # Adjust threshold as needed

def catboost_feature_selection(X, y):
    catboost = CatBoostRegressor(silent=True)
    catboost.fit(X, y)
    return X.columns[catboost.feature_importances_ > 0.01]  # Adjust threshold as needed

def svr_feature_selection(X, y):
    svr = SVR(kernel='linear')  # Use a linear kernel
    svr.fit(X, y)
    return X.columns[svr.coef_[0] != 0]  # Select features with non-zero coefficients

In [30]:
# List of your datasets
datasets = [mean_mode_processed, median_mode_processed, knn_processed]
target_variable = 'content.onRoadPrice'

# Initialize a dictionary to hold selected features
selected_features_dict = {
    'Lasso': {},
    'Ridge': {},
    'ElasticNet': {},
    'RandomForest': {},
    'GradientBoosting': {},
    'XGBoost': {},
    'LightGBM': {},
    'CatBoost': {},
    'SVR': {}
}

# Apply feature selection for each dataset and each method
for i, df in enumerate(datasets):
    X = df.drop(columns=[target_variable])
    y = df[target_variable]

    selected_features_dict['Lasso'][f'Dataset_{i+1}'] = lasso_feature_selection(X, y)
    selected_features_dict['Ridge'][f'Dataset_{i+1}'] = ridge_feature_selection(X, y)
    selected_features_dict['ElasticNet'][f'Dataset_{i+1}'] = elastic_net_feature_selection(X, y)
    selected_features_dict['RandomForest'][f'Dataset_{i+1}'] = random_forest_feature_selection(X, y)
    selected_features_dict['GradientBoosting'][f'Dataset_{i+1}'] = gradient_boosting_feature_selection(X, y)
    selected_features_dict['XGBoost'][f'Dataset_{i+1}'] = xgboost_feature_selection(X, y)
    selected_features_dict['LightGBM'][f'Dataset_{i+1}'] = lightgbm_feature_selection(X, y)
    selected_features_dict['CatBoost'][f'Dataset_{i+1}'] = catboost_feature_selection(X, y)
    selected_features_dict['SVR'][f'Dataset_{i+1}'] = svr_feature_selection(X, y)

# Create separate DataFrames for each algorithm's results
filtered_dfs = {}

for method, results in selected_features_dict.items():
    for key, selected_features in results.items():
        # Create a DataFrame for each method and dataset combination
        filtered_dfs[f'{method}_{key}'] = datasets[int(key.split('_')[1]) - 1][selected_features]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000740 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1001
[LightGBM] [Info] Number of data points in the train set: 2719, number of used features: 103
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000867 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 998
[LightGBM] [Info] Number of data points in the train set: 2719, number of used features: 103
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] Found 

In [31]:
# Initialize an empty list to hold the DataFrames
filtered_columns_list = []

# Iterate through the filtered_dfs dictionary
for key, df in filtered_dfs.items():
    # Create a new DataFrame with the selected features
    temp_df = pd.DataFrame(df)
    
    # Optionally, add a column to identify the method and dataset
    temp_df['Method_Dataset'] = key
    
    # Append the DataFrame to the list
    filtered_columns_list.append(temp_df)

# Concatenate all the DataFrames into a single DataFrame
combined_filtered_df = pd.concat(filtered_columns_list, ignore_index=True)

In [38]:
# Assuming filtered_columns_list is your list of DataFrames
shapes = []
column_names = []

for df in filtered_columns_list:
    shapes.append(df.shape)  # Appending the shape (rows, columns)
    column_names.append(df.columns.tolist())  # Appending the column names as a list

# Creating a DataFrame with the shapes and column names
result_df = pd.DataFrame({
    'Shape': shapes,
    'Column Names': column_names
})

In [44]:
result_df[['Rows', 'Columns']] = pd.DataFrame(result_df['Shape'].tolist(), index=result_df.index)

In [46]:
result_df.drop(columns=['Shape'], inplace=True)

In [51]:
result_df = result_df.sort_values('Columns')

In [57]:
result_df

,Column Names,Rows,Columns
11,"[Airbags, Bootspacelitres, MaxPowerbhp, MaxTor...",2719,6
10,"[Airbags, ABSAntilockBrakingSystem, Bootspacel...",2719,7
9,"[Airbags, ABSAntilockBrakingSystem, Bootspacel...",2719,7
13,"[Airbags, ABSAntilockBrakingSystem, FueltankCa...",2719,8
14,"[Airbags, ABSAntilockBrakingSystem, Bootspacel...",2719,8
16,"[Airbags, ABSAntilockBrakingSystem, FueltankCa...",2719,9
12,"[Airbags, ABSAntilockBrakingSystem, Bootspacel...",2719,9
17,"[Airbags, ABSAntilockBrakingSystem, MaxPowerbh...",2719,10
15,"[Airbags, ABSAntilockBrakingSystem, FueltankCa...",2719,12
2,"[content.year, Airbags, ABSAntilockBrakingSyst...",2719,18


### All unique column names from above dataframe

In [58]:
# Step 1: Flatten the lists in 'Column Names'
all_column_names = [name for sublist in result_df['Column Names'] for name in sublist]

# Step 2: Get unique column names using a set
unique_column_names = set(all_column_names)

# Convert back to a list if needed
unique_column_names = list(unique_column_names)

In [61]:
sorted(unique_column_names)

['ABSAntilockBrakingSystem',
 'AirConditioner',
 'Airbags',
 'Bootspacelitres',
 'Displacementcc',
 'FueltankCapacitylitres',
 'GroundClearancemm',
 'Left Front Tyre_WARN',
 'Left Rear Tyre_WARN',
 'MaxPowerbhp',
 'MaxPowerrpm',
 'MaxTorqueNm',
 'Method_Dataset',
 'Right Front Tyre_WARN',
 'Right Rear Tyre_WARN',
 'SeatingCapacity',
 'Spare Tyre_WARN',
 'content.bodyType_SUV',
 'content.bodyType_Sedan',
 'content.duplicateKey',
 'content.fitnessUpto_months_remaining',
 'content.fuelType_Diesel',
 'content.fuelType_Petrol',
 'content.insuranceExpiry_months_remaining',
 'content.insuranceType_Comprehensive',
 'content.insuranceType_Zero Depreciation',
 'content.make_Ford',
 'content.make_Honda',
 'content.make_Hyundai',
 'content.make_Jeep',
 'content.make_KIA',
 'content.make_MG',
 'content.make_Mahindra',
 'content.make_Maruti',
 'content.make_Nissan',
 'content.make_Renault',
 'content.make_Skoda',
 'content.make_Tata',
 'content.make_Toyota',
 'content.make_Volkswagen',
 'content.mod

#### All unique column names which can be used for prediction

- ABSAntilockBrakingSystem
- AirConditioner
- Airbags
- Bootspacelitres
- Displacementcc
- FueltankCapacitylitres
- GroundClearancemm
- Tyre_WARN
- MaxPowerbhp
- MaxPowerrpm
- MaxTorqueNm
- Method_Dataset
- SeatingCapacity
- content.bodyType
- content.duplicateKey
- content.fitnessUpto_months_remaining
- content.fuelType
- content.insuranceExpiry_months_remaining
- content.insuranceType
- content.make
- content.model
- content.odometerReading
- content.ownerNumber
- content.transmission
- content.variant
- content.year
- defects
- repainted

### Final PreProcessing after feature selection which can be automated - Add these in PreProcessing

- merge & change Tyre warn to 1-2-3-4-5
- impute missing values using KNN
- perform OHE, Transformation & Scaling
- check & remove cols based on corelation b/w MaxPowerbhp, MaxPowerrpm, MaxTorqueNm